# Atividade 1 da Disciplica de Machine Learning
## Alunos - Matrícula: 
    Gabriel Miranda - 202100011430
    Rafael Takeguma -
    Arthur Santos   - 202000012560
## Etapa 1: Pré-processamento dos dados

Abrindo o dataset em um dataframe pandas

In [ ]:
import pandas as pd

df = pd.read_csv("dataset.csv")
df = df.dropna()
df.head()

Exibindo informações gerais sobre o dataset

In [ ]:
df.info()

Exibindo as colunas numéricas do dataset

In [ ]:
df.describe()

Inicialmente, removeremos colunas que não se mostram importantes para a predição da evasão, como: matrícula, data de nascimento, local de nascimento, id do estudante, centro e nome do departamento.
Outros campos podem ser removidos posteriormente, depois de análises mais detalhadas.

In [ ]:
df = df.drop(columns=['curriculum_id', 'student_id', 'centro', 'department_name', 'birthday', 'birth_place'])
df.head()

Agora, verificaremos a importância das colunas restantes utilizando o feature_importances_ do RandomForestClassifier.
Para os dados não-numéricos, devemos utilizar o one_hot_encoding (get_dummies) da própria biblioteca pandas, apra discretizar essas colunas.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Aplicação do one_hot_encoding nas colunas não-numéricas
numericas = df.select_dtypes(include=['int64', 'float64']).columns
nominal = df.select_dtypes(include=['object']).columns
nominal = nominal.drop(['leaving_reason'])
nominal_encoded_df = pd.get_dummies(df[nominal], drop_first=True)

df_encoded = pd.concat([df[numericas], nominal_encoded_df, df['leaving_reason']], axis=1)

# Separação do dataframe em features (somente numéricas) e target
X = df_encoded.drop(columns=['leaving_reason'])
y = df_encoded['leaving_reason']

X

In [ ]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X,y)

# Dataframe da importância
importancia = pd.DataFrame({'Variável': X.columns, 'Importância': clf.feature_importances_})
importancia = importancia.sort_values(by="Importância", ascending=False)
importancia

É possível observar que as colunas de locks_number pra baixo possuem pouca importância na predição do modelo (menos que 5%). Logo, removeremos essas colunas, visando simplificar o modelo.

In [ ]:
df_encoded = df_encoded.drop(columns=['locks_number', 'course_iepl', 'course_iech', 'course_mc', 'ingress_form_VESTIBULAR', 'course_name_ENGENHARIA DE COMPUTAÇÃO',
                                      'ingress_form_TRANSFERÊNCIA INTERNA', 'course_name_SISTEMAS DE INFORMAÇÃO', 'ingress_form_TRANSFERÊNCIA COMPULSÓRIA', 
                                      'ingress_form_TRANSFERÊNCIA VOLUNTÁRIA', 'ingress_form_READMISSÃO', 'ingress_form_SUB JUDICE'])
df_encoded.head()

Após remoção inicial, utilizamos a função hist() para vizualizar os dados numéricos em forma de histograma.

Dessa forma verificamos a distribuição dos dados e suas escalas.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

df_encoded.hist(bins=20, figsize=(20,15))
plt.show()

É possível observar que há uma descrepância na distibuição desses dados, com o iepl, por exemplo, indo de 0.3 até 1.1 enquanto o required_ch_progress vai de 0 a 100.
Portanto, devemos aplicar alguma técnica de normalização nesses dados.
Primeiro, separamos o dataset em treino e teste.

In [26]:
from sklearn.model_selection import train_test_split

X = df_encoded.drop(columns=['leaving_reason'])
y = df_encoded['leaving_reason']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Etapa 2 - Seleção de modelo
Verificando a acuracia do RandomForestClassifier que já foi treinado

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred_class = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_class)
print(f"Acurácia: {accuracy:.2f}")

print("Relatório de Classificação:")
print(classification_report(y_test, y_pred_class))

print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred_class))


Agora vamos converter y para valores numericos e usar a regressão linear:

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Normalizando os dados
scaler = StandardScaler()


# Codificação de 'leaving_reason' para números
label_encoder = LabelEncoder()
y_numeric = label_encoder.fit_transform(df['leaving_reason'])

# Separação do dataframe em features (somente numéricas) e target
X = df_encoded.drop(columns=['leaving_reason'])
y = y_numeric  # Agora 'y' é a versão numérica

# Separando os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustando e transformando os dados de treinamento
X_train_scaled = scaler.fit_transform(X_train)

# Transformando os dados de teste
X_test_scaled = scaler.transform(X_test)

# Criando o modelo
modelo = LinearRegression()

# Treinando o modelo
modelo.fit(X_train_scaled, y_train)

# Fazendo previsões
y_pred = modelo.predict(X_test_scaled)

# Calculando as métricas
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5
r2 = r2_score(y_test, y_pred)

# Exibindo os resultados
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2:.2f}")


Agora vamos usar DecisonTreeRegressor:

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler


# Normalizando os dados
scaler = StandardScaler()

# Ajustando e transformando os dados de treinamento
X_train_scaled = scaler.fit_transform(X_train)

# Transformando os dados de teste
X_test_scaled = scaler.transform(X_test)

# Criando o modelo de árvore de decisão para regressão
modelo_tree = DecisionTreeRegressor(random_state=42)

# Ajustando o modelo aos dados de treinamento
modelo_tree.fit(X_train_scaled, y_train)

# Fazendo previsões sobre os dados de teste
y_pred_tree = modelo_tree.predict(X_test_scaled)

# Calculando as métricas de avaliação
mse_tree = mean_squared_error(y_test, y_pred_tree)
rmse_tree = mse_tree**0.5
r2_tree = r2_score(y_test, y_pred_tree)

# Exibindo as métricas
print(f"Mean Squared Error (MSE): {mse_tree:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse_tree:.2f}")
print(f"R² Score: {r2_tree:.2f}")


Vamos testar agora o RandomForestetRegressor:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Normalizando os dados
scaler = StandardScaler()

# Ajustando e transformando os dados de treinamento
X_train_scaled = scaler.fit_transform(X_train)

# Transformando os dados de teste
X_test_scaled = scaler.transform(X_test)

# Instanciar o modelo RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Treinar o modelo
rf_model.fit(X_train_scaled, y_train)

# Fazer previsões
y_pred_rf = rf_model.predict(X_test_scaled)

# Calcular as métricas de avaliação
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = mse_rf**0.5
r2_rf = r2_score(y_test, y_pred_rf)

# Exibir os resultados
print(f"Random Forest - Mean Squared Error (MSE): {mse_rf:.2f}")
print(f"Random Forest - Root Mean Squared Error (RMSE): {rmse_rf:.2f}")
print(f"Random Forest - R² Score: {r2_rf:.2f}")

Vamos usar o KNeighborsRegressor agora:

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Normalizando os dados
scaler = StandardScaler()

# Ajustando e transformando os dados de treinamento
X_train_scaled = scaler.fit_transform(X_train)

# Transformando os dados de teste
X_test_scaled = scaler.transform(X_test)

# Instanciar o modelo KNeighborsRegressor
knn_model = KNeighborsRegressor(n_neighbors=5)

# Treinar o modelo
knn_model.fit(X_train_scaled, y_train)

# Fazer previsões
y_pred_knn = knn_model.predict(X_test_scaled)

# Calcular as métricas de avaliação
mse_knn = mean_squared_error(y_test, y_pred_knn)
rmse_knn = mse_knn**0.5
r2_knn = r2_score(y_test, y_pred_knn)

# Exibir os resultados
print(f"KNN - Mean Squared Error (MSE): {mse_knn:.2f}")
print(f"KNN - Root Mean Squared Error (RMSE): {rmse_knn:.2f}")
print(f"KNN - R² Score: {r2_knn:.2f}")


Agora vamos comparar os modelos:

In [46]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Normalização dos dados
scaler = StandardScaler()

# Ajustando e transformando os dados de treinamento
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Definindo os modelos
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "K-Nearest Neighbors": KNeighborsRegressor(n_neighbors=5),
    "Decision Tree": DecisionTreeRegressor(random_state=42)
}

# Dicionário para armazenar os resultados
results = {}

# Treinando e avaliando os modelos
for name, model in models.items():
    model.fit(X_train_scaled, y_train)  # Treinando o modelo
    y_pred = model.predict(X_test_scaled)  # Fazendo previsões
    
    # Calculando as métricas
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    r2 = r2_score(y_test, y_pred)
    
    # Armazenando os resultados
    results[name] = {
        "MSE": mse,
        "RMSE": rmse,
        "R² Score": r2
    }

# Exibindo os resultados
for name, metrics in results.items():
    print(f"Modelo: {name}")
    print(f"Mean Squared Error (MSE): {metrics['MSE']:.2f}")
    print(f"Root Mean Squared Error (RMSE): {metrics['RMSE']:.2f}")
    print(f"R² Score: {metrics['R² Score']:.2f}")
    print("-" * 50)


Modelo: Linear Regression
Mean Squared Error (MSE): 4.96
Root Mean Squared Error (RMSE): 2.23
R² Score: 0.38
--------------------------------------------------
Modelo: Random Forest
Mean Squared Error (MSE): 4.35
Root Mean Squared Error (RMSE): 2.09
R² Score: 0.46
--------------------------------------------------
Modelo: K-Nearest Neighbors
Mean Squared Error (MSE): 4.86
Root Mean Squared Error (RMSE): 2.20
R² Score: 0.39
--------------------------------------------------
Modelo: Decision Tree
Mean Squared Error (MSE): 10.27
Root Mean Squared Error (RMSE): 3.20
R² Score: -0.28
--------------------------------------------------


De acordo com os resultados o modelo com o melhor resultado foi o RandomForestRegressor.